# Manual model testing

This notebook contains a number of tests for the simulation model where the output must be interpreted by a user to determine if the model is functioning correctly.   This may include inspecting:

* a simulated log of events
* the distribution of an output
* a chart or plot produced by the model

## Imports

In [1]:
import numpy as np
import statistics

## Model imports

In [2]:
from stroke_rehab_model import *

## Results collection tests

In [3]:
def test_results_collection_moments_1(audit_interval=1):
    '''
    Test ASU ward occupancy data collected is in a sensible range.
    
    Expected result: The type collected is int. The values are in 
    the range in the range 0 to 50 with sensible moments.

    prints out:
    min
    max
    mean
    deciles

    Params:
    ------
    audit_interval: 1
        duration of audit.

    Returns:
    -------
    None
    '''
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model#
    # set ASU arr~ival rates
    default_experiment_params = Experiment()

    # modified iteration 19. 
    # add RU, but do not run the model
    rehab_unit = RehabilitationUnit(env, default_experiment_params)
    
    acu_experiment = AcuteStrokeUnit(env, default_experiment_params, rehab_unit)
    
    # Start the patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())

    # optional results collection
    # Start the audit_acute_occupancy generator function to record ASU occupancy at intervals
    # MODIFIED iter 21
    env.process(audit_acute_occupancy(env, 1, audit_interval, acu_experiment, default_experiment_params))
    
    # Run the simulation until the specified run length in the Experiment parameters
    # MODIFIED iter 21
    env.run(until=default_experiment_params.results_collection_period)

    print(f'{min(default_experiment_params.asu_occupancy)=}')
    print(f'{max(default_experiment_params.asu_occupancy)=}')
    print(f'{statistics.fmean(default_experiment_params.asu_occupancy)=}')
    print([round(q, 1) for q in statistics.quantiles(default_experiment_params.asu_occupancy, n=10)])

In [4]:
def test_results_collection_moments_2(audit_interval=1):
    '''
    Test REHAB ward occupancy data collected is in a sensible range.
    
    Expected result: The type collected is int. The values are in 
    the range in the range 1 to [10-15] with sensible moments.

    Note this is when the Rehab unit is used independently
    with external arrivals only i.e. no transfers from ASU.

    prints out:
    min
    max
    mean
    deciles

    Params:
    ------
    audit_interval: 1
        duration of audit.

    Returns:
    -------
    None
    '''
    # Create the simulation environment
    env = simpy.Environment()
    
    # create experiment
    default_experiment = Experiment()
    
    rehab_unit = RehabilitationUnit(env, default_experiment)

    # Initialise the patient generators as simpy processes
    env.process(rehab_unit.stroke_patient_generator())
    env.process(rehab_unit.neuro_patient_generator())
    env.process(rehab_unit.other_patient_generator())
    
    # Initialise the audit of rehab occupancy as a simpy process with an interval of 1 day
    # MODIFIED iter 12
    env.process(audit_rehab_occupancy(env, 1, audit_interval, rehab_unit, default_experiment))
    
    # Run the model for the default run length in the experiment
    # MODIFIED iter 12
    env.run(until=default_experiment.results_collection_period)
    
    print(f'{min(default_experiment.rehab_occupancy)=}')
    print(f'{max(default_experiment.rehab_occupancy)=}')
    print(f'{statistics.fmean(default_experiment.rehab_occupancy)=}')
    print([round(q, 1) for q in statistics.quantiles(default_experiment.rehab_occupancy, n=10)])

In [5]:
def test_results_collection_moments_3(audit_interval=1):
    '''
    SYSTEM_TEST
    
    Test the model collects acute stroke occupancy every day

    Expected result: The type collected is int. The values are in 
    the range in the range 1 to ~25 with sensible moments.

    Note this is when the Rehab unit is linked to the ASU.
    I.e. transfers from ASU + external arrivals

    prints out:
    min
    max
    mean
    deciles

    Params:
    ------
    audit_interval: 1
        duration of audit.

    Returns:
    -------
    bool: does the model pass the test.
    '''
    # Create the simulation environment
    env = simpy.Environment()
    
    # a default experiment
    default_experiment_params = Experiment()

    # create models
    rehab_unit = RehabilitationUnit(env, default_experiment_params)
    acu_experiment = AcuteStrokeUnit(env, default_experiment_params, rehab_unit)
    
    # Start the ASU patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())

    # Initialize the pREHAB atient generators as simpy processes
    env.process(rehab_unit.stroke_patient_generator())
    env.process(rehab_unit.neuro_patient_generator())
    env.process(rehab_unit.other_patient_generator())  

    # optional results collection
    # Start the audit_acute_occupancy generator function to record ASU occupancy at intervals
    # MODIFIED iter 12
    env.process(audit_acute_occupancy(env, 1, audit_interval, acu_experiment, default_experiment_params))

    # Initialize the audit of rehab occupancy as a simpy process with an interval of 1 day
    # MODIFIED iter 12
    env.process(audit_rehab_occupancy(env, 1, audit_interval, rehab_unit, default_experiment_params))
    
    # Run the simulation until the specified run length in the Experiment parameters
    # MODIFIED iter 12
    env.run(until=default_experiment_params.results_collection_period * 10)

    # test
    print(f'{min(default_experiment_params.rehab_occupancy)=}')
    print(f'{max(default_experiment_params.rehab_occupancy)=}')
    print(f'{statistics.fmean(default_experiment_params.rehab_occupancy)=}')
    print([round(q, 1) for q in statistics.quantiles(default_experiment_params.rehab_occupancy, n=10)])

## Test suppress trace

These tests check that the simulated log can be suppressed when the model is running

In [6]:
def test_suppress_log_asu(trace):
    '''
    Test that setting experiment.trace 
    False suppresses output.

    Expected result: patient_count > 0

    Params:
    ------
    trace: bool
        is trace outputted or not?

    Returns:
    -------
    int: number of patient arrivals.
    '''
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model#
    # set ASU arr~ival rates
    default_experiment_params = Experiment(
        trace=trace,
    )

    # modified iteration 19. 
    # add RU, but do not run the model
    rehab_unit = RehabilitationUnit(env, default_experiment_params)
    
    acu_experiment = AcuteStrokeUnit(env, default_experiment_params, rehab_unit)
    
    # Start the patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())
    
    # Run the simulation until the specified run length in the Experiment parameters
    # MODIFIED iter 12
    env.run(until=default_experiment_params.results_collection_period)

    return acu_experiment.patient_count

In [7]:
def test_suppress_log_rehab(trace):
    '''
    Test that setting experiment.trace to
    False suppresses output in the rehab model

    Expected result: patient_count > 0

    Params:
    ------
    trace: bool
        is trace outputted or not?

    Returns:
    -------
    int: number of patient arrivals.
    '''
    # Create the simulation environment
    env = simpy.Environment()

    # create experiment
    default_experiment_params = Experiment(

        trace=trace,   
    )

    # this will be need to be modified in subsequent iterations
    rehab_unit = RehabilitationUnit(env, default_experiment_params)
    
    # Start the patient generators for each type of patient in the instance
    env.process(rehab_unit.stroke_patient_generator())
    env.process(rehab_unit.neuro_patient_generator())
    env.process(rehab_unit.other_patient_generator())
        
    # Run the simulation until the specified run length
    env.run(until=100)

    # print out stats collected
    print(f'{rehab_unit.patient_count=}')
    print(f'{rehab_unit.stroke_count=}')
    print(f'{rehab_unit.neuro_count=}')
    print(f'{rehab_unit.other_count=}')
    print(f'{rehab_unit.occupancy=}')
    return rehab_unit.patient_count

### Extreme value tests

Extreme value tests are used pragmatically to block of routes/arrivals/activites in the simulation model and check the results.

The most simple way to modify the model for these tests is to set parameters to $M$ a very large number.

In [15]:
def ev_test_1(large_number):
    '''
    TIA, Complex Neuro, Other, have their inter-arrival 
    time is set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model 
    is "Stroke". This is seen in the event log

    Params:
    -------
    large_number: int
        M a very large number 

    Returns:
    --------
    int: the number of patients that arrived to the model.
    '''
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model#
    # set ASU arrival rates
    default_experiment_params = Experiment(
        tia_mean=large_number,
        neuro_mean=large_number,
        other_mean=large_number,
        trace=True,
    )

    # modified iteration 19. 
    # add RU, but do not run the model
    rehab_unit = RehabilitationUnit(env, default_experiment_params)
    
    acu_experiment = AcuteStrokeUnit(env, default_experiment_params, rehab_unit)
    
    # Start the patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())
    
    # Run the simulation until the specified run length in the Experiment parameters
    # MODIFIED iter 12
    env.run(until=default_experiment_params.results_collection_period)

    return acu_experiment.patient_count

## Run tests

### Results collection moments 1 (ASU only)

In [8]:
test_results_collection_moments_1()

min(default_experiment_params.asu_occupancy)=1
max(default_experiment_params.asu_occupancy)=21
statistics.fmean(default_experiment_params.asu_occupancy)=8.296600877192983
[5.0, 6.0, 7.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0]


### Results collection moments 2 (REHAB only)

In [9]:
test_results_collection_moments_2()

min(default_experiment.rehab_occupancy)=0
max(default_experiment.rehab_occupancy)=8
statistics.fmean(default_experiment.rehab_occupancy)=2.633771929824561
[1.0, 1.0, 2.0, 2.0, 2.0, 3.0, 3.0, 4.0, 5.0]


### Results collection moment 3 (ASU+REHAB)

In [10]:
test_results_collection_moments_3()

min(default_experiment_params.rehab_occupancy)=0
max(default_experiment_params.rehab_occupancy)=21
statistics.fmean(default_experiment_params.rehab_occupancy)=9.399145158638829
[6.0, 7.0, 8.0, 9.0, 9.0, 10.0, 11.0, 12.0, 13.0]


### Test suppress trace
The following tests should only output statistics after the model run. There should be no printed log from within the run.

In [11]:
test_suppress_log_asu(trace=False)

2821

In [13]:
test_suppress_log_rehab(trace=False)

rehab_unit.patient_count=8
rehab_unit.stroke_count=4
rehab_unit.neuro_count=1
rehab_unit.other_count=3
rehab_unit.occupancy=0


8

### Extreme value tests

In [17]:
M = 10_000_000

In [18]:
ev_test_1(M)

Stroke patient 1 arrived at 0.03448962093012004 days and will go to Rehab
Stroke patient 2 arrived at 0.4406268799281962 days and will go to Rehab
Stroke patient 3 arrived at 0.6509676624994637 days and will go to Other
Stroke patient 4 arrived at 1.2314453050357161 days and will go to Other
Stroke patient 5 arrived at 1.7226581551516447 days and will go to Rehab
Stroke patient 6 arrived at 2.033109034369018 days and will go to Other
Stroke patient 6 finished treatment at 2.7931678527037325 days
Stroke patient 7 arrived at 3.668084847597135 days and will go to ESD
Stroke patient 7 finished treatment at 4.752089497603479 days
Stroke patient 8 arrived at 4.979194982801859 days and will go to Other
Stroke patient 8 finished treatment at 5.28654643841973 days
Stroke patient 8 finished treatment at 5.892784326292065 days
Stroke patient 9 arrived at 6.0023654986365615 days and will go to ESD
Stroke patient 9 finished treatment at 6.996000454412489 days
Stroke patient 9 finished treatment at 

1525